<a href="https://colab.research.google.com/github/donghui-0126/mini-project/blob/main/shoes-project/resell_regressor/img_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold


warnings.filterwarnings('ignore')     

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
df = pd.read_csv('/content/drive/MyDrive/kream_data/product_data_dpp.csv')
df = df[['img_path', 'brand', 'price_resell']]
df.head()

,img_path,brand,price_resell
0,crawling\product_crawling\image\28029.jpg,Nike,152800
1,crawling\product_crawling\image\12831.jpg,Nike,136200
2,crawling\product_crawling\image\21935.jpg,Nike,173200
3,crawling\product_crawling\image\44653.jpg,Nike,281400
4,crawling\product_crawling\image\89548.jpg,Nike,258000


In [7]:
drive_img_path_list = []

for index,data in enumerate(df['img_path']):
    drive_img_path = "/content/drive/MyDrive/kream_data/image/" + (data.split("\\")[-1])
    drive_img_path_list.append(drive_img_path)

In [8]:
df["drive_img_path"] = drive_img_path_list

In [9]:
df.head(3)

,img_path,brand,price_resell,drive_img_path
0,crawling\product_crawling\image\28029.jpg,Nike,152800,/content/drive/MyDrive/kream_data/image/28029.jpg
1,crawling\product_crawling\image\12831.jpg,Nike,136200,/content/drive/MyDrive/kream_data/image/12831.jpg
2,crawling\product_crawling\image\21935.jpg,Nike,173200,/content/drive/MyDrive/kream_data/image/21935.jpg


In [ ]:
pixel_list = []
for n,url in enumerate(df['drive_img_path']):
    img_name = (url)
    im = Image.open(img_name)
    pix = np.array(im)

    print("\r{}".format(n), end="")

    pixel_list.append(pix)
df['pixel'] = pixel_list

In [ ]:
df["pixel"].head()

0    [[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
1    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
2    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
3    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
4    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
Name: pixel, dtype: object

In [ ]:
df.head(3)

In [ ]:
# df.to_csv("/content/drive/MyDrive/kream_data/product_data_dpp.csv")

In [13]:
# pixel_data= np.array(df["pixel"])

In [ ]:
# pd.DataFrame(pixel_data).head()

In [ ]:
# pixel = np.concatenate(pixel_data, axis=0).reshape((6103,256,256,1))

In [ ]:
# pixel.shape

In [ ]:
# np.save('/content/drive/MyDrive/kream_data/pixel_data', pixel)

In [148]:
df = pd.read_csv("/content/drive/MyDrive/kream_data/product_data_dpp.csv")

In [149]:
pixel = np.load('/content/drive/MyDrive/kream_data/pixel_data.npy')
pixel.shape

(6103, 256, 256, 1)

In [150]:
pixel = pixel/255.

In [151]:
pixel = pixel.astype(np.float32)

In [152]:
nike_index = df[df['brand']=='Nike'].index
Jordan_index = df[df['brand']=='Jordan'].index

In [159]:
nike_X = pixel[nike_index]
Jordan_X = pixel[Jordan_index]

nike_y = df.loc[nike_index,'price_resell'] / 10000
Jordan_y = df.loc[Jordan_index,'price_resell'] / 10000


In [160]:
from sklearn.model_selection import train_test_split

nike_train_X, nike_test_X, nike_train_y, nike_test_y = train_test_split(nike_X, nike_y, test_size=0.3, shuffle=True)
nike_valid_X, nike_test_X, nike_valid_y, nike_test_y = train_test_split(nike_test_X, nike_test_y, test_size=0.5)

jordan_train_X, jordan_test_X, jordan_train_y, jordan_test_y = train_test_split(Jordan_X, Jordan_y, test_size=0.3, shuffle=True)
jordan_valid_X, jordan_test_X, jordan_valid_y, jordan_test_y = train_test_split(jordan_test_X, jordan_test_y, test_size=0.5)

In [161]:
%cd /content/drive/MyDrive/kream_data/model

/content/drive/MyDrive/kream_data/model


In [162]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc_nike=ModelCheckpoint('nike_img_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
mc_jordan=ModelCheckpoint('jordan_img_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)

In [163]:
input_shape=(256,256, 1)

nike_img_model = keras.models.Sequential([
    (keras.layers.Conv2D(32, (3, 3), input_shape=input_shape))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    ,(keras.layers.Conv2D(32, (3, 3)))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    ,(keras.layers.Conv2D(64, (3, 3)))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    ,(keras.layers.Flatten())
    ,(keras.layers.Dense(64))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.Dropout(0.5))
    ,(keras.layers.Dense(1))
    ])

nike_img_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss=tf.keras.losses.Huber(delta=3), metrics= ["mean_absolute_percentage_error"])

nike_img_model.fit(nike_train_X, nike_train_y, batch_size=16, epochs=300, validation_data = (nike_valid_X,nike_valid_y),  callbacks=[es, mc_nike])

Epoch 1/300
64/64 [==============================] - ETA: 0s - loss: 73.0467 - mean_absolute_percentage_error: 104.2977
Epoch 1: val_loss improved from inf to 64.87867, saving model to nike_img_best_model.h5
64/64 [==============================] - 9s 115ms/step - loss: 73.0467 - mean_absolute_percentage_error: 104.2977 - val_loss: 64.8787 - val_mean_absolute_percentage_error: 89.3609
Epoch 2/300
63/64 [============================>.] - ETA: 0s - loss: 59.6832 - mean_absolute_percentage_error: 78.3190
Epoch 2: val_loss improved from 64.87867 to 50.05983, saving model to nike_img_best_model.h5
64/64 [==============================] - 2s 30ms/step - loss: 59.5230 - mean_absolute_percentage_error: 78.0881 - val_loss: 50.0598 - val_mean_absolute_percentage_error: 61.9152
Epoch 3/300
64/64 [==============================] - ETA: 0s - loss: 40.9000 - mean_absolute_percentage_error: 47.2555
Epoch 3: val_loss improved from 50.05983 to 32.47134, saving model to nike_img_best_model.h5
64/64 [===

In [164]:
model = keras.models.load_model('nike_img_best_model.h5', custom_objects=None, compile=True)

model.evaluate(nike_test_X, nike_test_y)

7/7 [==============================] - 1s 61ms/step - loss: 38.5838 - mean_absolute_percentage_error: 42.0992


[38.583770751953125, 42.0992431640625]

In [165]:
input_shape=(256,256, 1)

nike_img_model = keras.models.Sequential([
    (keras.layers.Conv2D(32, (3, 3), input_shape=input_shape))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    ,(keras.layers.Conv2D(32, (3, 3)))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    ,(keras.layers.Conv2D(64, (3, 3)))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    ,(keras.layers.Flatten())
    ,(keras.layers.Dense(64))
    ,(keras.layers.Activation('relu'))
    ,(keras.layers.Dropout(0.5))
    ,(keras.layers.Dense(1))
    ])

nike_img_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss=tf.keras.losses.Huber(delta=3), metrics= ["mean_absolute_percentage_error"])

nike_img_model.fit(jordan_train_X, jordan_train_y, batch_size=16, epochs=300, validation_data = (jordan_valid_X,jordan_valid_y),  callbacks=[es, mc_jordan])

Epoch 1/300
61/61 [==============================] - ETA: 0s - loss: 59.6094 - mean_absolute_percentage_error: 56.5363
Epoch 1: val_loss improved from inf to 46.45196, saving model to jordan_img_best_model.h5
61/61 [==============================] - 5s 46ms/step - loss: 59.6094 - mean_absolute_percentage_error: 56.5363 - val_loss: 46.4520 - val_mean_absolute_percentage_error: 50.6816
Epoch 2/300
58/61 [===========================>..] - ETA: 0s - loss: 51.9162 - mean_absolute_percentage_error: 46.2538
Epoch 2: val_loss did not improve from 46.45196
61/61 [==============================] - 1s 23ms/step - loss: 52.9868 - mean_absolute_percentage_error: 46.1074 - val_loss: 48.1928 - val_mean_absolute_percentage_error: 57.0675
Epoch 3/300
60/61 [============================>.] - ETA: 0s - loss: 53.3441 - mean_absolute_percentage_error: 47.1060
Epoch 3: val_loss improved from 46.45196 to 45.62363, saving model to jordan_img_best_model.h5
61/61 [==============================] - 3s 45ms/step 

In [166]:
model = keras.models.load_model('jordan_img_best_model.h5', custom_objects=None, compile=True)

model.evaluate(jordan_test_X, jordan_test_y)

7/7 [==============================] - 0s 43ms/step - loss: 27.7482 - mean_absolute_percentage_error: 40.7805


[27.748199462890625, 40.780494689941406]